In [ ]:
import os

from pandas import DataFrame, date_range
import numpy as np

from idealreport.settings import load_settings
from idealreport import create_html
from idealreport.reporter import Reporter
import htmltag

In [ ]:
# load settings from settings.hjson
settings = load_settings()

# instantiate reporter object and set output location
output_file = os.path.join(settings['output_path'], 'sample_plots.html')
r = Reporter(title='Sample Plots', output_file=output_file)

# report start
r.h += htmltag.h3('This report provides an overview of the different plot types.')

In [ ]:
# vertical bar chart

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3', 'Entity 4', 'Entity 5'],
                  'Stat 1': np.random.randn(5).tolist(),
                  'Stat 2': np.random.randn(5).tolist(),
                })

# note the x values are generally assumed to be in the index, with each non index column being a series of y values
# a few exceptions are outlined below
ch = ch.set_index('Entity')
ch = ch.sort_values(by='Stat 1', ascending=False)

# color choices for bars
markers = [
    dict(color = 'rgb(49, 105, 196)'),
    dict(color = 'rgb(195, 197, 201)')
]

r.h += htmltag.h4('Bar Charts')

# plotting calls are of the form r.plot.plot_type
# a look through the plotter class in reporter.py will show the available plot types and arguments for each
# in all single plot type cases, the only required argument is df, though for multi a types array with plot types is also required
r.plot.bar(df=ch, title='Vertical Bar Chart', xlabel='Entity', ylabel='%', markers=markers,)

In [ ]:
# stacked horizontal bar chart, notice additional entry in the data field for orientation

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3aaaaaaaaaaaaaaaa', 'Entity 4', 'Entity 5'],
                'Foo': np.random.randn(5).tolist(),
                'Bar': np.random.randn(5).tolist(),
               })
ch = ch.set_index('Entity')
ch = ch[['Foo', 'Bar']] # make sure columns have desired order

markers = [
    dict(color = 'rgb(49, 105, 196)'),
    dict(color = 'rgb(195, 197, 201)')
]

# stacked bars behave oddly with mixed positive / negative numbers, so only use when all numbers are positive or negative within a series
ch = ch.abs()
ch = ch*100

# all reporter plots take a layout argument which allows you to specify any element that would appear in plotly's api
# here we're adjusting the left margin of the graph (layout['margin']['l']) to 150 to make room for the long Entity 3 name.
r.plot.barh(df=ch, title='Horizontal Stacked Bar Chart', stacked=True, xlabel='$', markers=markers, layout={'margin':{'l':150}})

In [ ]:
# horizontal overlay bar chart

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3', 'Entity 4', 'Entity 5'],
                'Foo': np.random.randn(5).tolist(),
                'Bar': np.random.randn(5).tolist(),
               })
ch = ch.set_index('Entity')
ch = ch[['Foo', 'Bar']] # make sure columns have desired order

markers = [
    dict(color = 'rgb(195, 197, 201)'),
    dict(color = 'rgb(49, 105, 196)'),
]

# bar charts take in an argument called width which determines the size of the bars
# for line charts, shown below, this is controlled by an entry in the lines dict for width
# for scatter plots, this is controlled by an entry in the markers dict for size
widths = [.4, .2]

# the overlay bar charts also takes opacity an argument,
# generally the "focus" bar should be the second column so that it is drawn over the first bar
opacities = [.6, 1]

r.plot.baro(df=ch, title='Vertical Overlay Bar Chart', orientation='v', xlabel='$', markers=markers, widths=widths, opacities=opacities)

In [ ]:
# histogram, note that we don't use the index for x values here because there is no x value other than the series

ch = DataFrame({'Foo': np.random.randn(50).tolist(),
                'Bar': np.random.randn(50).tolist(),
               })
# offset Bar so that there's two different distros to observe
ch['Bar'] = ch['Bar'] + 2

markers = [
    dict(color = 'rgb(49, 105, 196)'),
    dict(color = 'rgb(195, 197, 201)')
]

r.plot.histo(df=ch, title='Histogram', ylabel='Observations', markers=markers)

In [ ]:
# line chart

ch = DataFrame(np.random.randn(20, 3))
ch.columns = ['a', 'b', 'c']
ch.a = range(20)
ch = ch.set_index('a')

# as mentioned above, the lines are governed by a lines attribute instead of markers, width is set in the dict
lines = [
    dict(width = 7, color = 'rgb(49, 105, 196)'),
    dict(width = 3, color = 'rgb(195, 197, 201)')
]

r.h += htmltag.h4('Line Charts')
r.plot.line(df=ch, title='Line Chart', xlabel='Entity', ylabel='%', lines=lines)

In [ ]:
# time series

ind = date_range('2017-11-02 9:00', periods=20, freq='T')
ch = DataFrame(data=np.random.randn(20,2), index=ind)
ch.columns = ['a', 'b']

lines = [
    dict(width = 7, color = 'rgb(49, 105, 196)'),
    dict(width = 3, color = 'rgb(195, 197, 201)')
]

# just like a regular line except the index is timestamps
r.plot.time(df=ch, title='Time Series', xlabel='Time', ylabel='%', lines=lines)

In [ ]:
# scatter plot

ch = DataFrame(np.random.randn(20, 3))
ch.columns = ['a', 'b', 'c']
ch = ch.set_index('a')

# marker sizes can also be set like the line widths, though the parameter name is different
markers = [
    dict(size = 7, color = 'rgb(49, 105, 196)'),
    dict(size = 3, color = 'rgb(195, 197, 201)')
]

# example of a more customized layout
layout = dict(font=dict(family='Arial', color='#77797c'),
    xaxis=dict(
        showgrid=False,
        showline=False,
        zerolinecolor='#acadaf',
        ticks='outside',
        tickcolor='#acadaf',
        hoverformat='.2f',
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=False,
        zerolinecolor='#acadaf',
        linewidth=10,
        ticks='',
        tickcolor='#acadaf',
        hoverformat='.2f',
    ),
    width=500,
    height=500,
    showlegend=False,
)

r.h += htmltag.h4('Scatter Plots')
r.plot.scatter(df=ch, title='Scatter Plot', xlabel='alpha', ylabel='beta', markers=markers, layout=layout)

In [ ]:
# univariate plots
# note these are just scatter plots with x or y values of 0 and the mode of markers+text (and accompanying text and text position)

ch = DataFrame(np.random.randn(1,5))
ch.columns = ['a', 'b', 'c', 'd', 'e']
ch = ch + 5
# note index (x) value is 0

# here we use data_static, which has attributes which are the same across each trace (column of a df), note these can be arrays
data_static={'mode':'markers+text', 'textposition':'right',}
# here we use data_to_iterate, which has attributes which differ by trace (column of a df), note these must be arrays, but can be arrays of arrays or dicts
data_to_iterate={'text':['a', 'b', 'c', 'd', 'e']}

# for vertical univariate plots, turn off the xaxis autotick and set height/width appropriately
layout = dict(
    xaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        showline=False,
        zerolinecolor='#acadaf',
        linewidth=10,
        autotick=False,
        showticklabels=False        
    ),
    yaxis=dict(
        autorange=True,
        showgrid=False,
        showline=False,
        zerolinecolor='#acadaf',
        ticks='',
        tickcolor='#acadaf',
        showticklabels=True
    ),
    autosize=False,
    width=250,
    height=500,
    showlegend=False
)

r.plot.scatter(df=ch, title='Vertical Univariate Plot', xlabel='alpha', data_static=data_static, data_to_iterate=data_to_iterate, layout=layout)

# flip xaxis / yaxis attriutes and height/width for horizontal version, pass through orientation flag to flip x and y values as well
(layout['xaxis'], layout['yaxis']) = (layout['yaxis'], layout['xaxis'])
(layout['width'], layout['height']) = (layout['height'], layout['width'])
data_static.update({'orientation':'h', 'textposition':'top'})

r.plot.scatter(df=ch, title='Horizontal Univariate Plot', xlabel='alpha', data_static=data_static, data_to_iterate=data_to_iterate, layout=layout)

In [ ]:
# mixed plot types on one graph

ch1 = DataFrame(np.random.randn(20, 2))
ch1.columns = ['a', 'b1']
ch1.sort_values(by='a', ascending=True, inplace=True)
ch1 = ch1.set_index('a')

ch2 = DataFrame(np.random.randn(20, 2))
ch2.columns = ['a', 'b2']
ch2.sort_values(by='a', ascending=True, inplace=True)
ch2 = ch2.set_index('a')

# set lines attributes, which should have an appropriate entry for each chart type passed through
# note the None second element corresponding to the bar chart which doesn't need a lines attribute
lines = [[dict(color = 'rgb(49, 105, 196)')],None]

# set markers attributes, which should have an appropriate entry for each chart type passed through
# note the leading None element corresponding to the line chart which doesn't need a markers attribute
markers = [None,[dict(color = 'rgb(195, 197, 201)')]]

r.h += htmltag.h4('Mixed Type Plots')
r.plot.multi(dfs=[ch1, ch2], types=['line', 'bar'], title='Mixed Line and Bar Plot', xlabel='x label', ylabel='y label', lines=lines, markers=markers)

In [ ]:
# different axes

ch1 = DataFrame(np.random.randn(20, 2))
ch1.columns = ['x', 'a']
ch1.x = range(20)
ch1 = ch1.set_index('x')

ch2 = DataFrame(np.random.randn(20, 2))
ch2.columns = ['x', 'b']
ch2.x = range(20)
ch2 = ch2.set_index('x')
ch2.b *= 100

# given the y2 axis, move the legend to the bottom of the plot
layout={'legend':{'orientation':'h', 'x':0.8, 'y':-0.1}}

# y2_axis is a list of booleans indicating whether each df should be tied to the y2 axis or not
r.plot.multi(dfs=[ch1, ch2], types=['line', 'line'], title='Lines with Different Y Axes', xlabel='x', ylabel='ya', y2_axis=[False,True], y2label='yb', layout=layout)

In [ ]:
# ohlc

ch = DataFrame(np.random.randn(4, 4))
ch.columns = ['open', 'high', 'low', 'close']
ch['high'] = ch['high'] + 3
ch['low'] = ch['low'] - 3

ch['dealer'] = range(4)
ch = ch.set_index('dealer')

r.h += htmltag.h4('Open High Low Close (OHCL) Plot')
r.plot.ohlc(df=ch, title='OHLC Plot by Reporter', xlabel='price', ylabel='instrument')

In [ ]:
# pie

ch = DataFrame(data=[1,2,3,4], index=['Factor %s' % x for x in range(4)])

r.h += htmltag.h4('Pie Charts')
r.plot.pie(df=ch, title='Pie')

layout = {'height':300}

# only difference for a donut is the hole argument
r.plot.pie(df=ch, title='Donut', hole=.4, layout=layout)

In [ ]:
# error bar charts

ch = DataFrame(np.random.randn(20, 4))
ch.columns = ['x','y','e1','e2']
ch = ch.set_index('x')
ch['e1'] = ch['e1'].abs()*0.1
ch['e2'] = ch['e2'].abs()*0.1

r.h += htmltag.h4('Error Bar Charts')
# note only first 3 columns are used for symmetric error bars
r.plot.errbar(df=ch, title='Symmetric Error Bars')

# just add the flag symmetric=False, otherwise it defaults to True like in the call above
r.plot.errbar(df=ch, title='Asymmetric Error Bars', symmetric=False)

In [ ]:
# error line plots

ch = DataFrame({'day': ['1', '2', '3', '4', '5', '6', '7', '8'],
                'mean': [15, 13, 7, 18, 19, 24, 25, 28],
                'stdev': [1, 2, 1.3, 1, 2.5, .5, 1.5, .75]})
ch = ch[['day', 'mean', 'stdev']] # make sure columns have desired order
ch = ch.set_index('day')

r.h += htmltag.h4('Error Line Plots')
r.plot.errline(df=ch, title='Symmetric Error Lines')

In [ ]:
# generate and save the report HTML
r.generate()